In [82]:
from __future__ import division
from scipy.integrate import quad
from copy import copy
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt


class demandFunc():
    
    def __init__(self,**params):
        for key in params:
                setattr(self, key, params[key])

    def demand(self, P):
        return self.multiplier*P**-self.elasticity
    
    def invDemand(self, Q):
        return (self.multiplier/Q)**(1/self.elasticity)

    def demandCurve(self, Pvec):
        '''
        Pvec is a vector of prices
        '''
        return map(self.demand, Pvec)
    
    def invDemandCurve(self, Qvec):
        '''
        Qcev is a vector of quantities
        '''
        return map(self.invDemand, Qvec)
        
        
class supplyFunc():

    def __init__(self,interval_points):
        '''
        interval points is a set of tupples (Q,P)
        '''
        self.interval_points = copy(interval_points)
        self.interval_points.sort(reverse=True)
        
    def supply(self,P):
        '''
        interval points are sorted from highest Quantity to lowest
        '''
        highest_q = self.interval_points[0][0]
        for q,p in self.interval_points:
            if p > P:
                highest_q = q
                continue
            if p <= P:
                return highest_q
        return highest_q
            
    def invSupply(self, Q):
        '''
        interval points are sorted from highest Quantity to lowest
        '''
        
        highest_p = self.interval_points[0][1]
        for q,p in self.interval_points:
            if q < Q:
                return p
            if q >= Q:
                highest_p = p
                continue
        return highest_p

    def supplyCurve(self, Pvec):
        '''
        Pvec is a vector of prices
        '''
        return map(self.supply, Pvec)
    
    def invSupplyCurve(self, Qvec):
        '''
        Qvec is a vector of quantities
        '''
        return map(self.invSupply, Qvec)
    
class market(object):
    
    def __init__(self, demandFunc, supplyFunc):
        self.demandFunc, self.supplyFunc = demandFunc, supplyFunc
        '''
        if ad < az:
            raise ValueError('Insufficient demand.')
        '''
        self.findEQ()
        
    def findEQ(self):
        "Return equilibrium price"
        ranges = ((1,10),(1,10))
        optimalFunc = lambda p : np.abs(self.demandFunc.invDemand(p[0]) - self.supplyFunc.invSupply(p[0]))
        results = optimize.brute(optimalFunc, ranges, full_output=True,
                              finish=optimize.fmin)
        self.equilibriumQ = results[0][0]
        self.equilibriumP = self.demandFunc.invDemand(self.equilibriumQ)

    '''    
    def consumer_surp(self):
        "Compute consumer surplus"
        # == Compute area under inverse demand function == #
        integrand = lambda x: (self.ad/self.bd) - (1/self.bd)* x
        area, error = quad(integrand, 0, self.quantity())
        return area - self.price() * self.quantity()
    
    def producer_surp(self):
        "Compute producer surplus"
        #  == Compute area above inverse supply curve, excluding tax == #
        integrand = lambda x: -(self.az/self.bz) + (1/self.bz) * x
        area, error = quad(integrand, 0, self.quantity())  
        return (self.price() - self.tax) * self.quantity() - area
    
    def taxrev(self):
        "Compute tax revenue"
        return self.tax * self.quantity()
        
    def inverse_demand(self,x):
        "Compute inverse demand"
        return self.ad/self.bd - (1/self.bd)* x
    
    def inverse_supply(self,x):
        "Compute inverse supply curve"
        return -(self.az/self.bz) + (1/self.bz) * x + self.tax
    
    def inverse_supply_no_tax(self,x):
        "Compute inverse supply curve without tax"
        return -(self.az/self.bz) + (1/self.bz) * x
    '''

In [98]:
stepPoints = [(0,2),(3,4),(6,9)]
demand_paramaters = {'multiplier':7,'elasticity':.7}
supplyFunction = supplyFunc(stepPoints)
demandFunction = demandFunc(**demand_paramaters)

Market = market(demandFunction,supplyFunction)
print(Market.equilibriumP)
print(Market.equilibriumQ)


3.35491191367
3.0


In [99]:


titlefont = {
        'family' : 'serif',
        'color'  : 'black',
        'weight' : 'bold',
        'size'   : 16,
        }

labelfont = {
        'family' : 'sans-serif',  # (cursive, fantasy, monospace, serif)
        'color'  : 'black',       # html hex or colour name
        'weight' : 'normal',      # (normal, bold, bolder, lighter)
        'size'   : 14,            # default value:12
        }


grid = np.linspace(1,10,50)
fig, ax = plt.subplots()
ax.plot(grid, demandFunction.invDemandCurve(grid), lw=2, alpha=0.6, label='demand')
ax.plot(grid, supplyFunction.invSupplyCurve(grid), lw=2, alpha=0.6, label='supply')

plt.title('Water Supply and Demand', fontdict=titlefont) 
plt.xlabel('Quantity of Water Demanded', fontdict=labelfont)
plt.ylabel('Price ($/mcm)', fontdict=labelfont)
ax.legend(loc='upper right', frameon=False, fontsize=14)
plt.show()

